## Descrição do problema

Temos que otimizar uma carteira de 12 investimentos, buscando minimizar o risco.

Quais então são os objetos ?

- Ativo::Float = o quanto temos para investir ao total
- Carteira::Agente <: Dict{Label::String ∪ Symbol, Investimento::Float} = Nossa carteira de investimentos:
  - O investimento é dividido do Ativo;
  - Teremos 12 investimentos;
- População::Array{Agente} = População que será sujeita ao algoritmo evo

In [1]:
import bovespa as bv
import deap as dp



In [2]:
from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [3]:
import random
from deap import tools

IND_SIZE = 12

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

/home/marcelovmaciel/miniconda3/lib/python3.7/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [4]:
toolbox

In [5]:
def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [8]:
from tqdm import tqdm
    
def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 200

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in tqdm(range(NGEN)):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop

In [9]:
main()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 715.29it/s]


[[-14.769356900626843,
  -22.42133995343212,
  -14.348365063303605,
  -17.969991578358957,
  -14.292749073391796,
  -18.459667845059617,
  -25.054935607636132,
  -16.85185114098054,
  -17.712898727535006,
  -21.635660153637154,
  -15.240201994956927,
  -15.366488039067333],
 [-14.769356900626843,
  -22.42133995343212,
  -14.348365063303605,
  -17.969991578358957,
  -14.292749073391796,
  -18.459667845059617,
  -25.054935607636132,
  -16.85185114098054,
  -17.712898727535006,
  -20.6121984590088,
  -15.240201994956927,
  -15.366488039067333],
 [-14.769356900626843,
  -24.341706995859248,
  -14.348365063303605,
  -17.428848948081445,
  -13.658240533607422,
  -19.70997141103366,
  -27.187290417695525,
  -16.85185114098054,
  -17.712898727535006,
  -21.635660153637154,
  -15.240201994956927,
  -15.366488039067333],
 [-14.720111842441852,
  -24.341706995859248,
  -14.348365063303605,
  -17.428848948081445,
  -13.462424926310517,
  -19.70997141103366,
  -25.6101504755255,
  -16.8518511409805